In [1]:
import pandas as pd
import numpy as np
import math

import time

In [2]:
# Importando .csv
train = pd.read_csv('dataset/train_data.csv')

In [ ]:
# Implementando baseline
def baseline(train):
    mean = train["rating"].mean(skipna = True)
    users = train.pivot(index='movie_id', columns='user_id', values='rating')
    items = train.pivot(index='user_id', columns='movie_id', values='rating')
    nusers = train.values[:, 0].max()+1
    nitems = train.values[:, 1].max()+1
    bu = np.full(nusers, np.nan, dtype=float)
    bi = np.full(nitems, np.nan, dtype=float)

    for i in items.columns.values:
        bi[i] = np.nanmean(items[i] - mean)

    aux = bi[~np.isnan(bi)]
    for u in users.columns.values:
        bu[u] = np.nanmean(users[u] - mean - aux)

    bi = np.nan_to_num(bi)
    bu = np.nan_to_num(bu)
    
    return { "bu" : bu, "bi": bi, "mean": mean }

In [4]:
# Predizendo nota
def predict(model, u, i):
    return model["mean"] + model["bu"][u] + model["bi"][i]

In [5]:
# Avaliando predições
def rmse(model, test):
    sum_err = 0
    for t in test:
        u = t[0]
        i = t[1]
        r_ui = t[2]
        pred = predict(model, u, i)
        error = (r_ui - pred)**2
        sum_err += error
    return math.sqrt(sum_err/len(test))

In [40]:
def cross_validation(model, data, k_folds = 10):
    folds = np.array_split(data.sample(frac=1), k_folds)
    for i in range(k_folds):
        train = folds.copy()
        test = folds[i].values
        del train[i]
        train = pd.concat(train, sort=False)
        model_k = model(train)
        print("Iteração {0}: {1}".format(i+1, rmse(model_k, test)))

In [6]:
# baseline = baseline(train)

In [7]:
# rmse(baseline, train.values[401838:])

0.890356975214376

In [41]:
cross_validation(baseline, train, 10)

Iteração 1: 0.9027558085833594
Iteração 2: 0.9068414791638052
Iteração 3: 0.9036722227396531
Iteração 4: 0.9153573260784718
Iteração 5: 0.9080129395767453
Iteração 6: 0.9080010566823896
Iteração 7: 0.9039993390907823
Iteração 8: 0.9049222919249483
Iteração 9: 0.9089968233744676
Iteração 10: 0.9032049150467893
